In [2]:
import pandas as pd
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
# Veriyi birleştirme ve ön işleme
file_path = './Datasets/*.csv'
all_files = glob.glob(file_path)
df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

combined_df = pd.concat(df_list)
combined_df.sort_values(by='timestamp', inplace=True)
output_file = 'combined.csv'
combined_df.to_csv(output_file, index=False)

data_k = pd.read_csv('combined.csv')

In [4]:
# Sadece gerekli kolonları seçelim
data_k_filtered = data_k[['timestamp', 'sensor', 'pm25_avg_60']]
data_k_filtered['timestamp'] = pd.to_datetime(data_k_filtered['timestamp'])

# Saatlik ortalama
hourly_avg = data_k_filtered.resample('H', on='timestamp').mean().reset_index()

# Rüzgar verisi ekleme
wind = pd.read_csv('fresno_wind.csv')
wind['datetime'] = pd.to_datetime(wind['datetime']).dt.tz_localize('UTC')

# Verileri birleştirme
merged_data = pd.merge(hourly_avg, wind[['datetime', 'windspeed', 'winddir']], left_on='timestamp', right_on='datetime',
                       how='inner')
merged_data = merged_data.drop(columns=['datetime'])

/var/folders/vr/11d0bknx2ms1fxq4p99sr8tc0000gn/T/ipykernel_11112/2170410342.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_k_filtered['timestamp'] = pd.to_datetime(data_k_filtered['timestamp'])
/var/folders/vr/11d0bknx2ms1fxq4p99sr8tc0000gn/T/ipykernel_11112/2170410342.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hourly_avg = data_k_filtered.resample('H', on='timestamp').mean().reset_index()


In [5]:
# Veriyi normalleştirme fonksiyonu
def normalise(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return pd.DataFrame(X_scaled, columns=X.columns)

In [6]:
# Özellikleri normalleştir
features = ['pm25_avg_60', 'windspeed', 'winddir']
merged_data[features] = normalise(merged_data[features])

In [15]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2924 entries, 0 to 2923
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   timestamp    2924 non-null   datetime64[ns, UTC]
 1   pm25_avg_60  2924 non-null   float64            
 2   windspeed    2924 non-null   float64            
 3   winddir      2924 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(3)
memory usage: 114.2 KB


In [8]:
# Anomali üretim sınıfı
class AnomalyGenerator:
    def __init__(self, model):
        self.model = model

    def generate_fgsm(self, data, eps=0.1):
        # FGSM kullanarak anomaliler oluştur
        adv_data = data.clone().detach()
        noise = eps * torch.sign(torch.randn_like(data))  # Rastgele gürültü ekle
        adv_data += noise
        adv_data = torch.clamp(adv_data, 0, 1)  # 0 ve 1 arasında kısıtla
        return adv_data

    def generate_bim(self, data, eps=0.1, alpha=0.01, steps=10):
        # BIM kullanarak anomaliler oluştur
        adv_data = data.clone().detach()
        for _ in range(steps):
            adv_data.requires_grad = True
            outputs = self.model(adv_data)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            adv_data += alpha * adv_data.grad.sign()
            adv_data = torch.clamp(adv_data, 0, 1)  # 0 ve 1 arasında kısıtla
            adv_data.grad.zero_()
        return adv_data

    def generate_pgd(self, data, eps=0.1, alpha=0.01, steps=10):
        # PGD kullanarak anomaliler oluştur
        adv_data = data.clone().detach()
        for _ in range(steps):
            adv_data.requires_grad = True
            outputs = self.model(adv_data)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            adv_data = adv_data.detach() + alpha * adv_data.grad.sign()
            delta = torch.clamp(adv_data - data, min=-eps, max=eps)
            adv_data = torch.clamp(data + delta, min=0, max=1).detach()
            adv_data.grad.zero_()
        return adv_data

In [9]:
# Veriyi 28:4:9 oranında ayırma
train_size = 0.28
val_size = 0.04
test_size = 0.09

# İlk olarak, train setini ayırıyoruz.
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, train_size=train_size, random_state=42)

# Daha sonra kalan veriyi validation ve test seti olarak ayırıyoruz.
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size/(test_size + val_size), random_state=42)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# GraPhy katmanı tanımlama
class GraPhyLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraPhyLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        return out

# GraPhy Modeli
class GraPhyModel(nn.Module):
    def __init__(self, input_size, hidden_dim=512, output_size=2):
        super(GraPhyModel, self).__init__()
        self.layer1 = GraPhyLayer(input_size, hidden_dim)
        self.layer2 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer3 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer4 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer5 = GraPhyLayer(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc_out(x)
        return x

# Modeli oluşturma
input_size = X_train.shape[1]
model = GraPhyModel(input_size)

# Adam optimizer'ı ve parametreleri
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

# Kayıp fonksiyonu
criterion = nn.CrossEntropyLoss()

In [11]:
from torch.utils.data import DataLoader, TensorDataset

# Tensor Dataset ve DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)

# Eğitim döngüsü
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Her epoch sonunda doğrulama seti ile değerlendirme
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch [1/50], Loss: 12.2682, Validation Loss: 4.4748
Epoch [2/50], Loss: 6.0266, Validation Loss: 4.3963
Epoch [3/50], Loss: 5.5184, Validation Loss: 4.3354
Epoch [4/50], Loss: 5.5581, Validation Loss: 4.2708
Epoch [5/50], Loss: 5.3445, Validation Loss: 4.2451
Epoch [6/50], Loss: 5.3636, Validation Loss: 4.2887
Epoch [7/50], Loss: 5.2052, Validation Loss: 4.2075
Epoch [8/50], Loss: 5.2350, Validation Loss: 4.1990
Epoch [9/50], Loss: 5.3892, Validation Loss: 4.1421
Epoch [10/50], Loss: 5.2214, Validation Loss: 4.1043
Epoch [11/50], Loss: 5.1814, Validation Loss: 4.1141
Epoch [12/50], Loss: 5.1718, Validation Loss: 4.1037
Epoch [13/50], Loss: 5.1611, Validation Loss: 4.1176
Epoch [14/50], Loss: 5.2026, Validation Loss: 4.1623
Epoch [15/50], Loss: 5.1363, Validation Loss: 4.0717
Epoch [16/50], Loss: 5.0201, Validation Loss: 4.0594
Epoch [17/50], Loss: 5.1317, Validation Loss: 4.0794
Epoch [18/50], Loss: 5.0604, Validation Loss: 4.0745
Epoch [19/50], Loss: 5.1062, Validation Loss: 4.1288
E

In [12]:
# Test setinde modeli değerlendirme
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 94.79%


In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Modeli test setinde değerlendirme
correct = 0
total = 0
all_labels = []
all_predictions = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Precision, Recall, F1-Score ve Accuracy hesaplama
accuracy = accuracy_score(all_labels, all_predictions)
report = classification_report(all_labels, all_predictions, target_names=['Normal', 'Anomali'])

# Confusion Matrix ile specificity hesaplama
conf_matrix = confusion_matrix(all_labels, all_predictions)
tn, fp, fn, tp = conf_matrix.ravel()

specificity = tn / (tn + fp)  # Özgüllük (Specificity)
precision = tp / (tp + fp)    # Doğruluk (Precision)
recall = tp / (tp + fn)       # Duyarlılık (Recall)
f1 = 2 * (precision * recall) / (precision + recall)  # F1 Skoru

# Sonuçları yazdırma
print("Classification Report:\n", report)
print(f"Accuracy: {accuracy:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Classification Report:
               precision    recall  f1-score   support

      Normal       0.95      1.00      0.97      1382
     Anomali       0.00      0.00      0.00        76

    accuracy                           0.95      1458
   macro avg       0.47      0.50      0.49      1458
weighted avg       0.90      0.95      0.92      1458

Accuracy: 0.9479
Specificity: 1.0000
Precision: nan
Recall: 0.0000
F1 Score: nan


/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/var/folders/vr/11d0bknx2ms1fxq4p99sr

In [ ]:
# Anomali yaratma sınıfı
class AnomalyGenerator:
    def __init__(self, model, eps=0.1, alpha=0.01, steps=10, random_start=True):
        self.model = model
        self.eps = eps
        self.alpha = alpha
        self.steps = steps
        self.random_start = random_start

    def fgsm(self, data, labels):
        data = data.clone().detach().requires_grad_(True)
        outputs = self.model(data)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        grad = torch.autograd.grad(loss, data)[0]
        adv_data = data + self.eps * grad.sign()
        return torch.clamp(adv_data, min=0, max=1)

    def generate_anomalies(self, data, labels, method='fgsm'):
        if method == 'fgsm':
            return self.fgsm(data, labels)
        else:
            raise ValueError(f"Unknown method: {method}")

In [ ]:
# Anomaliler üretme (isteğe bağlı)
anomaly_gen = AnomalyGenerator(model)
X_test_adv = anomaly_gen.generate_anomalies(X_test, y_test)

In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

# Veri setini yükle
# merged_data = pd.read_csv('your_data.csv') # Veriyi yükleyin
# Özellikleri normalleştir
def normalise(data):
    return (data - data.mean()) / data.std()

features = ['pm25_avg_60', 'windspeed', 'winddir']
merged_data[features] = normalise(merged_data[features])

# Eğitim, doğrulama ve test setlerine ayır
X = merged_data[features].values
y = np.zeros(merged_data.shape[0], dtype=int)
num_anomalies = int(0.05 * merged_data.shape[0])
anomaly_indices = np.random.choice(merged_data.index, num_anomalies, replace=False)
y[anomaly_indices] = 1  # 1: Anomali

# Veriyi eğitim, doğrulama ve test setlerine ayır
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.13, random_state=42)  # %28 eğitim
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6923, random_state=42)  # %4 doğrulama ve %9 test

In [17]:
# GraPhy katmanı tanımlama
class GraPhyLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraPhyLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        return out

# GraPhy Modeli
class GraPhyModel(nn.Module):
    def __init__(self, input_size, hidden_dim=512, output_size=2):
        super(GraPhyModel, self).__init__()
        self.layer1 = GraPhyLayer(input_size, hidden_dim)
        self.layer2 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer3 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer4 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer5 = GraPhyLayer(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc_out(x)
        return x

In [18]:
class AnomalyGenerator:
    def __init__(self, model):
        self.model = model

    def generate_fgsm(self, data, labels, eps=0.1):
        data = data.clone().detach().requires_grad_(True)
        outputs = self.model(data)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        self.model.zero_grad()
        loss.backward()

        # FGSM ile anomali üret
        adv_data = data + eps * data.grad.sign()
        return adv_data.detach()

    def generate_bim(self, data, labels, eps=0.1, alpha=0.01, steps=10):
        data = data.clone().detach()
        ori_data = data.clone().detach()

        for _ in range(steps):
            data.requires_grad = True
            outputs = self.model(data)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            self.model.zero_grad()
            loss.backward()

            data = data + alpha * data.grad.sign()
            data = torch.clamp(data, ori_data - eps, ori_data + eps).detach()

        return data

    def generate_pgd(self, data, labels, eps=0.1, alpha=0.01, steps=10, random_start=True):
        data = data.clone().detach()
        if random_start:
            data = data + torch.empty_like(data).uniform_(-eps, eps)
            data = torch.clamp(data, min=0, max=1).detach()

        ori_data = data.clone().detach()

        for _ in range(steps):
            data.requires_grad = True
            outputs = self.model(data)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            self.model.zero_grad()
            loss.backward()

            data = data + alpha * data.grad.sign()
            data = torch.clamp(data, ori_data - eps, ori_data + eps).detach()

        return data

In [19]:
# Modeli oluşturma
input_size = X_train.shape[1]
model = GraPhyModel(input_size)

# Adam optimizer'ı ve kayıp fonksiyonu
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

# Tensor Dataset ve DataLoader
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)

# Eğitim döngüsü
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Her epoch sonunda doğrulama seti ile değerlendirme
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch [1/50], Loss: 24.1026, Validation Loss: 1.5193
Epoch [2/50], Loss: 15.6634, Validation Loss: 1.3401
Epoch [3/50], Loss: 15.5176, Validation Loss: 1.4785
Epoch [4/50], Loss: 15.0240, Validation Loss: 1.3777
Epoch [5/50], Loss: 14.8722, Validation Loss: 1.3524
Epoch [6/50], Loss: 14.8079, Validation Loss: 1.2901
Epoch [7/50], Loss: 14.6921, Validation Loss: 1.3838
Epoch [8/50], Loss: 14.7178, Validation Loss: 1.4183
Epoch [9/50], Loss: 14.8644, Validation Loss: 1.3729
Epoch [10/50], Loss: 14.6545, Validation Loss: 1.3491
Epoch [11/50], Loss: 14.7580, Validation Loss: 1.3560
Epoch [12/50], Loss: 14.6857, Validation Loss: 1.4932
Epoch [13/50], Loss: 14.6414, Validation Loss: 1.4618
Epoch [14/50], Loss: 14.5706, Validation Loss: 1.4367
Epoch [15/50], Loss: 14.4608, Validation Loss: 1.3983
Epoch [16/50], Loss: 14.4717, Validation Loss: 1.4505
Epoch [17/50], Loss: 14.5682, Validation Loss: 1.4234
Epoch [18/50], Loss: 14.3284, Validation Loss: 1.5200
Epoch [19/50], Loss: 14.4042, Validat

In [20]:
# Anomali verilerini üret
anomaly_generator = AnomalyGenerator(model)

# FGSM ile anomali üret
fgsm_anomalies = anomaly_generator.generate_fgsm(torch.FloatTensor(X_train), torch.LongTensor(y_train))

# BIM ile anomali üret
bim_anomalies = anomaly_generator.generate_bim(torch.FloatTensor(X_train), torch.LongTensor(y_train))

# PGD ile anomali üret
pgd_anomalies = anomaly_generator.generate_pgd(torch.FloatTensor(X_train), torch.LongTensor(y_train))

# Anomali verilerini test setine ekleyin
X_test_with_anomalies = np.vstack((X_test, fgsm_anomalies.numpy(), bim_anomalies.numpy(), pgd_anomalies.numpy()))
y_test_with_anomalies = np.hstack((y_test, np.ones(fgsm_anomalies.shape[0] + bim_anomalies.shape[0] + pgd_anomalies.shape[0])))

# Test setini oluştur
X_test_tensor = torch.FloatTensor(X_test_with_anomalies)
y_test_tensor = torch.LongTensor(y_test_with_anomalies)

# Modelin test edilmesi
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)

# Sonuçları değerlendir
accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
precision = precision_score(y_test_tensor.numpy(), predicted.numpy())
recall = recall_score(y_test_tensor.numpy(), predicted.numpy())
tn, fp, fn, tp = confusion_matrix(y_test_tensor.numpy(), predicted.numpy()).ravel()
specificity = tn / (tn + fp)
f1 = 2 * (precision * recall) / (precision + recall)

# Metrikleri yazdır
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.0310
Precision: 0.0000
Recall: 0.0000
Specificity: 1.0000
F1 Score: nan


/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/var/folders/vr/11d0bknx2ms1fxq4p99sr8tc0000gn/T/ipykernel_11112/2242473391.py:33: RuntimeWarning: invalid value encountered in scalar divide
  f1 = 2 * (precision * recall) / (precision + recall)


In [29]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2924 entries, 0 to 2923
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   timestamp    2924 non-null   datetime64[ns, UTC]
 1   pm25_avg_60  2924 non-null   float64            
 2   windspeed    2924 non-null   float64            
 3   winddir      2924 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(3)
memory usage: 178.8 KB


In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

# Veri setini yükle
# merged_data = pd.read_csv('your_data.csv') # Veriyi yükleyin

# Özellikleri normalleştir
def normalise(data):
    return (data - data.mean()) / data.std()

features = ['pm25_avg_60', 'windspeed', 'winddir']
merged_data[features] = normalise(merged_data[features])

In [22]:
# Anomali verilerini üret
def generate_anomalies(data, num_anomalies=1000):
    anomalies = []
    for _ in range(num_anomalies):
        index = np.random.randint(0, data.shape[0])
        anomaly = data[index] + np.random.normal(0, 1, data[index].shape)  # Normal dağılımdan gürültü ekleyerek anomali üret
        anomalies.append(anomaly)
    return np.array(anomalies)

# Normal verileri ayır
normal_data = merged_data[features].values

# Anomalileri üret
anomaly_data = generate_anomalies(normal_data, num_anomalies=1000)  # 1000 anomali üret

In [24]:
X = np.vstack((normal_data, anomaly_data))
y = np.hstack((np.zeros(normal_data.shape[0]), np.ones(anomaly_data.shape[0])))  # 0: Normal, 1: Anomali

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.13, random_state=42)  # %28 eğitim
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6923, random_state=42)  # %4 doğrulama ve %9 test

In [25]:
class GraPhyLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraPhyLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        return out

class GraPhyModel(nn.Module):
    def __init__(self, input_size, hidden_dim=512, output_size=2):
        super(GraPhyModel, self).__init__()
        self.layer1 = GraPhyLayer(input_size, hidden_dim)
        self.layer2 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer3 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer4 = GraPhyLayer(hidden_dim, hidden_dim)
        self.layer5 = GraPhyLayer(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc_out(x)
        return x

In [26]:
input_size = X_train.shape[1]
model = GraPhyModel(input_size)

optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch [1/50], Loss: 60.1515, Validation Loss: 2.5585
Epoch [2/50], Loss: 44.4334, Validation Loss: 2.1748
Epoch [3/50], Loss: 37.2413, Validation Loss: 1.8297
Epoch [4/50], Loss: 34.1474, Validation Loss: 1.7889
Epoch [5/50], Loss: 31.9695, Validation Loss: 1.6694
Epoch [6/50], Loss: 30.1265, Validation Loss: 1.6336
Epoch [7/50], Loss: 29.5877, Validation Loss: 1.5462
Epoch [8/50], Loss: 29.4214, Validation Loss: 1.4780
Epoch [9/50], Loss: 28.1360, Validation Loss: 1.6396
Epoch [10/50], Loss: 28.2332, Validation Loss: 1.5894
Epoch [11/50], Loss: 27.0595, Validation Loss: 1.4489
Epoch [12/50], Loss: 26.7501, Validation Loss: 1.5674
Epoch [13/50], Loss: 26.2023, Validation Loss: 1.5061
Epoch [14/50], Loss: 26.1530, Validation Loss: 1.5522
Epoch [15/50], Loss: 25.7509, Validation Loss: 1.5736
Epoch [16/50], Loss: 26.2591, Validation Loss: 1.4641
Epoch [17/50], Loss: 25.2213, Validation Loss: 1.4762
Epoch [18/50], Loss: 24.6848, Validation Loss: 1.4933
Epoch [19/50], Loss: 24.2464, Validat

In [27]:
model.eval()
with torch.no_grad():
    test_outputs = model(torch.FloatTensor(X_test))
    _, predicted = torch.max(test_outputs, 1)

accuracy = accuracy_score(y_test, predicted.numpy())
precision = precision_score(y_test, predicted.numpy())
recall = recall_score(y_test, predicted.numpy())
tn, fp, fn, tp = confusion_matrix(y_test, predicted.numpy()).ravel()
specificity = tn / (tn + fp)
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.8898
Precision: 0.8684
Recall: 0.6947
Specificity: 0.9614
F1 Score: 0.7719


In [28]:
model.eval()
with torch.no_grad():
    test_inputs = torch.tensor(X_test, dtype=torch.float32)
    test_outputs = model(test_inputs)

    _, predicted = torch.max(test_outputs, 1)

total_anomalies = np.sum(y_test)  
correct_predictions = np.sum(predicted.numpy()[y_test == 1])  # Modelin doğru tahmin ettiği anomaliler

print(f"Toplam gerçek anomali sayısı: {total_anomalies}")
print(f"Modelin doğru tahmin ettiği anomali sayısı: {correct_predictions}")
print(f"Modelin doğruluk oranı: {correct_predictions / total_anomalies * 100:.2f}%")

Toplam gerçek anomali sayısı: 95.0
Modelin doğru tahmin ettiği anomali sayısı: 66
Modelin doğruluk oranı: 69.47%
